In [1]:
# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs


In [2]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
toronto_task2_csv = "Toronto.TASK_II_df.csv"

In [6]:
df_toronto = pd.read_csv(toronto_task2_csv)
df_toronto.head(7)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029


In [7]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto.save("my_map1.html" )

In [8]:
from IPython.display import HTML

HTML('<iframe src=my_map1.html width=700 height=450></iframe>')

C:\Users\dell\anaconda3\lib\site-packages\IPython\core\display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [9]:
# @hiddel_cell
CLIENT_ID = 'FPIENWUBAFUUMXHPL5QK0LZICLXB1YAPUAHHPQCPVU3CUJB2' # your Foursquare ID
CLIENT_SECRET = 'CJBSSJYWDAUKOG40C5CFOJVYVILMAHI2AYPNBZGUWDX0Y1VJ' # your Foursquare Secret
VERSION = '20190530' # Foursquare API version

In [10]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029


In [11]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [12]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge,Malvern" are 43.8066863, -79.19435340000003.


In [13]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60dc8e9e927a316dc459356c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 35,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c648dbdf07e2d7fc4aa8f50',
       'name': 'Hot Topic',
       'location': {'address': '300 Borough Dr.',
        'crossStreet': 'at Scarborough Town Centre',
        'lat': 43.775450021914345,
        'lng': -79.2579286531551,
       

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

<ipython-input-16-9d4491af7edd>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Hot Topic,Clothing Store,43.775450,-79.257929
1,New York Fries - Scarborough TC,Restaurant,43.775135,-79.256858
2,Tommy Hilfiger,Clothing Store,43.776015,-79.257369
3,lululemon athletica,Clothing Store,43.775517,-79.258423
4,Subway,Sandwich Place,43.775102,-79.257115
5,Disney Store,Toy / Game Store,43.775537,-79.256833
6,Subway,Sandwich Place,43.771477,-79.252805
7,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
8,Petro-Canada,Gas Station,43.769724,-79.261677
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

35 venues were returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


In [20]:

scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
6,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
7,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
8,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Krispy Kreme Doughnuts,43.767169,-79.189660,Donut Shop
9,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


In [21]:

scarborough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
84,L'Amoreaux West,43.799525,-79.318389,KFC,43.798938,-79.318854,Fast Food Restaurant
85,L'Amoreaux West,43.799525,-79.318389,Shoppers Drug Mart,43.799966,-79.317985,Pharmacy
86,L'Amoreaux West,43.799525,-79.318389,Pizza Pizza,43.797909,-79.318113,Pizza Place
87,L'Amoreaux West,43.799525,-79.318389,Tim Hortons,43.799102,-79.318715,Coffee Shop
88,L'Amoreaux West,43.799525,-79.318389,McDonald's,43.798249,-79.318167,Fast Food Restaurant
89,L'Amoreaux West,43.799525,-79.318389,Metro,43.797830,-79.318492,Supermarket
90,L'Amoreaux West,43.799525,-79.318389,Eggsmart,43.796375,-79.318681,Breakfast Spot
91,L'Amoreaux West,43.799525,-79.318389,Fit4Less,43.798394,-79.318453,Gym
92,L'Amoreaux West,43.799525,-79.318389,Rogers Plus,43.798911,-79.318277,Electronics Store
93,L'Amoreaux West,43.799525,-79.318389,Ocean Nails Spa,43.795290,-79.320101,Nail Salon


In [22]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",4,4,4,4,4,4
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea,Golden Mile,Oakridge",10,10,10,10,10,10
"Clarks Corners,Sullivan,Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest,Cliffside,Scarborough Village West",3,3,3,3,3,3
"Dorset Park,Scarborough Town Centre,Wexford Heights",6,6,6,6,6,6
"East Birchmount Park,Ionview,Kennedy Park",6,6,6,6,6,6


In [23]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 58 uniques categories.


In [24]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Arts & Crafts Store,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Gym,Hakka Restaurant,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:

scarb_onehot.shape

(94, 59)

In [26]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,American Restaurant,Arts & Crafts Store,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Gym,Hakka Restaurant,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant,Women's Store
0,Agincourt,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0,0.00,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.25,0.0,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Birch Cliff,Cliffside West",0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.25,0.000000,0.000000,0.00,0.000000,0.25,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0
3,Cedarbrae,0.000000,0.00,0.142857,0.0,0.142857,0.142857,0.0,0.00,0.0,0.0,0.00,0.142857,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.142857,0.00,0.0,0.142857,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.142857,0.0,0.0,0.0
4,"Clairlea,Golden Mile,Oakridge",0.000000,0.00,0.000000,0.0,0.200000,0.000000,0.0,0.00,0.2,0.1,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.1,0.0,0.100000,0.000000,0.0,0.00,0.0,0.00,0.0,0.1,0.0,0.0,0.000000,0.0,0.10,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.1,0.0,0.000000,0.0,0.0,0.0
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.000000,0.00,0.000000,0.0,0.000000,0.083333,0.0,0.00,0.0,0.0,0.00,0.000000,0.083333,0.00,0.083333,0.00,0.083333,0.0,0.0,0.0,0.0,0.083333,0.083333,0.083333,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.083333,0.083333,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.166667,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.083333,0.0,0.0,0.0
6,"Cliffcrest,Cliffside,Scarborough Village West",0.333333,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.333333,0.0,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0


In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Lounge,Clothing Store,Women's Store,College Stadium,Gym,General Entertainment,Gas Station,Fried Chicken Joint
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Arts & Crafts Store,Intersection,Playground,Park,Women's Store,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,"Birch Cliff,Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Women's Store,Coffee Shop,Gym,Gas Station,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Hakka Restaurant,Bakery,Caribbean Restaurant,Bank,Gas Station,Athletics & Sports,Thai Restaurant,General Entertainment,College Stadium,Fried Chicken Joint
4,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Metro Station,Ice Cream Shop,Soccer Field,Intersection,Park,Bus Station,Donut Shop,Convenience Store
5,"Clarks Corners,Sullivan,Tam O'Shanter",Pizza Place,Chinese Restaurant,Bank,Convenience Store,Fast Food Restaurant,Coffee Shop,Italian Restaurant,Intersection,Fried Chicken Joint,Gas Station
6,"Cliffcrest,Cliffside,Scarborough Village West",American Restaurant,Intersection,Motel,Hobby Shop,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
7,"Dorset Park,Scarborough Town Centre,Wexford He...",Indian Restaurant,Vietnamese Restaurant,Light Rail Station,Pet Store,Chinese Restaurant,Women's Store,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint
8,"East Birchmount Park,Ionview,Kennedy Park",Chinese Restaurant,Coffee Shop,Train Station,Discount Store,Bus Station,Hobby Shop,Gym,General Entertainment,Gas Station,Fried Chicken Joint
9,"Guildwood,Morningside,West Hill",Restaurant,Medical Center,Intersection,Bank,Electronics Store,Mexican Restaurant,Rental Car Location,Breakfast Spot,Donut Shop,Discount Store


# Run K Clusters

In [29]:

# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(14)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([0, 3, 0, 0, 0, 0, 3, 0, 0, 0])

In [30]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Women's Store,Coffee Shop,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Donut Shop
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3,Home Service,Bar,Coffee Shop,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0,Restaurant,Medical Center,Intersection,Bank,Electronics Store,Mexican Restaurant,Rental Car Location,Breakfast Spot,Donut Shop,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean BBQ Restaurant,Pharmacy,Women's Store,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Bakery,Caribbean Restaurant,Bank,Gas Station,Athletics & Sports,Thai Restaurant,General Entertainment,College Stadium,Fried Chicken Joint
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Women's Store,Playground,Cosmetics Shop,Clothing Store,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,3,Chinese Restaurant,Coffee Shop,Train Station,Discount Store,Bus Station,Hobby Shop,Gym,General Entertainment,Gas Station,Fried Chicken Joint
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,0,Bakery,Bus Line,Metro Station,Ice Cream Shop,Soccer Field,Intersection,Park,Bus Station,Donut Shop,Convenience Store
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,0,American Restaurant,Intersection,Motel,Hobby Shop,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,0,General Entertainment,Skating Rink,Café,College Stadium,Women's Store,Coffee Shop,Gym,Gas Station,Fried Chicken Joint,Fast Food Restaurant


In [31]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('my_map3.html')

In [32]:
from IPython.display import HTML

HTML('<iframe src=my_map3.html width=700 height=450></iframe>')

C:\Users\dell\anaconda3\lib\site-packages\IPython\core\display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [33]:

scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Women's Store,Coffee Shop,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Donut Shop
2,Scarborough,0,Restaurant,Medical Center,Intersection,Bank,Electronics Store,Mexican Restaurant,Rental Car Location,Breakfast Spot,Donut Shop,Discount Store
3,Scarborough,0,Coffee Shop,Korean BBQ Restaurant,Pharmacy,Women's Store,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Scarborough,0,Hakka Restaurant,Bakery,Caribbean Restaurant,Bank,Gas Station,Athletics & Sports,Thai Restaurant,General Entertainment,College Stadium,Fried Chicken Joint
5,Scarborough,0,Women's Store,Playground,Cosmetics Shop,Clothing Store,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
7,Scarborough,0,Bakery,Bus Line,Metro Station,Ice Cream Shop,Soccer Field,Intersection,Park,Bus Station,Donut Shop,Convenience Store
8,Scarborough,0,American Restaurant,Intersection,Motel,Hobby Shop,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,Scarborough,0,General Entertainment,Skating Rink,Café,College Stadium,Women's Store,Coffee Shop,Gym,Gas Station,Fried Chicken Joint,Fast Food Restaurant
11,Scarborough,0,Middle Eastern Restaurant,Vietnamese Restaurant,Auto Garage,Bakery,Sandwich Place,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
12,Scarborough,0,Latin American Restaurant,Breakfast Spot,Lounge,Clothing Store,Women's Store,College Stadium,Gym,General Entertainment,Gas Station,Fried Chicken Joint


In [34]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Light Rail Station,Pet Store,Chinese Restaurant,Women's Store,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint


In [35]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,2,Pizza Place,Chinese Restaurant,Bank,Convenience Store,Fast Food Restaurant,Coffee Shop,Italian Restaurant,Intersection,Fried Chicken Joint,Gas Station


In [36]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Home Service,Bar,Coffee Shop,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,Scarborough,3,Chinese Restaurant,Coffee Shop,Train Station,Discount Store,Bus Station,Hobby Shop,Gym,General Entertainment,Gas Station,Fried Chicken Joint


In [37]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Scarborough,4,Fast Food Restaurant,Nail Salon,Bank,Gym,Pharmacy,Pizza Place,Breakfast Spot,Coffee Shop,Chinese Restaurant,Sandwich Place
